## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar10
# from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.utils import to_categorical
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import itertools
import matplotlib.pyplot as plt
from PIL import Image

In [0]:
# img_arr is of shape (n, h, w, c)
def resize_image_arr(img_arr):
    from skimage.transform import resize
    x_resized_list = []
    for i in range(img_arr.shape[0]):
        img = img_arr[0]
        resized_img = resize(img, (224, 224))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)

In [21]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# x_train = resize_image_arr(x_train)
# x_test = resize_image_arr(x_test)

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
input_shape = x_train.shape[1:]

In [23]:
resnet50_model = ResNet50(include_top=False,
                          input_shape=input_shape,)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [24]:
resnet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
my_model = resnet50_model.output

In [0]:
my_model = Flatten()(my_model)

In [27]:
my_model = Dropout(0.5)(my_model)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [28]:
output_layer = Dense(10, activation='softmax', name='softmax')(my_model)

In [0]:
net_final = Model(inputs=resnet50_model.input, outputs=output_layer)

In [0]:
for layer in net_final.layers[:2]:
    layer.trainable = False
for layer in net_final.layers[2:]:
    layer.trainable = True

In [32]:
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
print(net_final.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [36]:
history = net_final.fit(x_train, y_train,
                    batch_size=64,
                    epochs=30,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = net_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 79s 2ms/step - loss: 3.1625 - acc: 0.1715 - val_loss: 2.1600 - val_acc: 0.2764
Epoch 2/30
50000/50000 [==============================] - 63s 1ms/step - loss: 2.0402 - acc: 0.3101 - val_loss: 1.7331 - val_acc: 0.4480
Epoch 3/30
50000/50000 [==============================] - 63s 1ms/step - loss: 1.6356 - acc: 0.4487 - val_loss: 1.4063 - val_acc: 0.5448
Epoch 4/30
50000/50000 [==============================] - 64s 1ms/step - loss: 1.3516 - acc: 0.5452 - val_loss: 1.1778 - val_acc: 0.6086
Epoch 5/30
50000/50000 [==============================] - 63s 1ms/step - loss: 1.1486 - acc: 0.6121 - val_loss: 1.0462 - val_acc: 0.6461
Epoch 6/30
50000/50000 [==============================] - 63s 1ms/step - loss: 1.0033 - acc: 0.6607 - val_loss: 0.9569 - val_acc: 0.6802
Epoch 7/30
50000/50000 [========